# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and y

In [15]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 

import re
import dill as pickle
import time
import datetime

import pickle

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# load data from database - change path accordingly
engine = create_engine('sqlite:///../data/data_db/disaster_responses.db')
df = pd.read_sql_table('DisasterResponses', engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26028 entries, 0 to 26027
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26028 non-null  int64 
 1   message                 26028 non-null  object
 2   original                10038 non-null  object
 3   genre                   26028 non-null  object
 4   related                 26028 non-null  int64 
 5   request                 26028 non-null  int64 
 6   offer                   26028 non-null  int64 
 7   aid_related             26028 non-null  int64 
 8   medical_help            26028 non-null  int64 
 9   medical_products        26028 non-null  int64 
 10  search_and_rescue       26028 non-null  int64 
 11  security                26028 non-null  int64 
 12  military                26028 non-null  int64 
 13  child_alone             26028 non-null  int64 
 14  water                   26028 non-null  int64 
 15  fo

In [5]:
# The messages will be the features, while the categories will be the target variables
X = df.message.to_numpy() 
y = df[df.columns[4:]].to_numpy()

In [6]:
# Check size of dataset
print("Complete feature dataset size = ", X.shape, "; complete target variables size = ", y.shape)

Complete feature dataset size =  (26028,) ; complete target variables size =  (26028, 36)


In [7]:
# Check what's in X, y
X[0], y[0]

('Weather update - a cold front from Cuba that could pass over Haiti',
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [8]:
# Finally, save the names of the category labels
category_names = list(df.columns[4:])

In [9]:
# Check categories
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens_raw = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens_raw if (word not in stopwords.words('english'))]

    return tokens

In [11]:
# Test function for the first 5 messages
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Split data in test/train


In [12]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
# Check size of datasets - default split is 75% train / 25% test
print("Training feature dataset size = ", X_train.shape, "; training target variables size = ", y_train.shape)
print("Testing feature dataset size = ", X_test.shape, "; testing target variables size = ", y_test.shape)

Training feature dataset size =  (19521,) ; training target variables size =  (19521, 36)
Testing feature dataset size =  (6507,) ; testing target variables size =  (6507, 36)


### 4. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 5.a. Train pipeline

In [19]:
# train classifier, keeping track of time
t0= time.clock()

model.fit(X_train, y_train);

t1 = time.clock() - t0

In [20]:
print('elapsed time = ', t1)

elapsed time =  381.021145


### 5.b. Load saved pipeline

In case the pipeline was saved previously you can load it here

In [ ]:
# Load dictionary 
filename = '../models/models_files/cv_trained_model.pkl'
model_dict = pickle.load(open(filename, 'rb'))

In [ ]:
# Get content
X_train = model_dict['X_train']
y_train = model_dict['y_train']
X_test = model_dict['X_test']
y_test = model_dict['y_test']
model = model_dict['model']

### 6. Test your pipeline
 - Predict outcomes based on test data
 - Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
# Predict on test data
y_pred = model.predict(X_test)

In [22]:
# Display example of results 
print(classification_report(y_test[:,0], y_pred[:,0]))

              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1568
           1       0.84      0.95      0.89      4939

    accuracy                           0.82      6507
   macro avg       0.77      0.68      0.70      6507
weighted avg       0.80      0.82      0.80      6507



Run the following cells to show the report for all the labels, on screen

In [23]:
# Iterate and print on screen and file
for ind_1 in range(y_pred.shape[1]):
    print('-----------------------------------------------------------------------------------------')
    print('Label = ', category_names[ind_1])
    c_rep = classification_report(y_test[:,ind_1], y_pred[:,ind_1], output_dict=True, zero_division=0)
    kk = list(c_rep.keys())
    for ind_2 in range(len(c_rep) - 3):
        print('Value = ', kk[ind_2], ': precision = ', "{:.2f}".format(c_rep[kk[ind_2]]['precision']),
              '; recall = ', "{:.2f}".format(c_rep[kk[ind_2]]['recall']), 
              '; f1-s =', "{:.2f}".format(c_rep[kk[ind_2]]['f1-score']),
              '; support =', c_rep[kk[ind_2]]['support'])

-----------------------------------------------------------------------------------------
Label =  related
Value =  0 : precision =  0.71 ; recall =  0.41 ; f1-s = 0.52 ; support = 1568
Value =  1 : precision =  0.84 ; recall =  0.95 ; f1-s = 0.89 ; support = 4939
-----------------------------------------------------------------------------------------
Label =  request
Value =  0 : precision =  0.91 ; recall =  0.98 ; f1-s = 0.94 ; support = 5412
Value =  1 : precision =  0.85 ; recall =  0.51 ; f1-s = 0.64 ; support = 1095
-----------------------------------------------------------------------------------------
Label =  offer
Value =  0 : precision =  1.00 ; recall =  1.00 ; f1-s = 1.00 ; support = 6479
Value =  1 : precision =  0.00 ; recall =  0.00 ; f1-s = 0.00 ; support = 28
-----------------------------------------------------------------------------------------
Label =  aid_related
Value =  0 : precision =  0.79 ; recall =  0.84 ; f1-s = 0.82 ; support = 3772
Value =  1 : precis

Value =  0 : precision =  0.99 ; recall =  1.00 ; f1-s = 0.99 ; support = 6434
Value =  1 : precision =  1.00 ; recall =  0.03 ; f1-s = 0.05 ; support = 73
-----------------------------------------------------------------------------------------
Label =  earthquake
Value =  0 : precision =  0.98 ; recall =  0.99 ; f1-s = 0.98 ; support = 5914
Value =  1 : precision =  0.90 ; recall =  0.78 ; f1-s = 0.83 ; support = 593
-----------------------------------------------------------------------------------------
Label =  cold
Value =  0 : precision =  0.98 ; recall =  1.00 ; f1-s = 0.99 ; support = 6387
Value =  1 : precision =  0.76 ; recall =  0.16 ; f1-s = 0.26 ; support = 120
-----------------------------------------------------------------------------------------
Label =  other_weather
Value =  0 : precision =  0.95 ; recall =  1.00 ; f1-s = 0.97 ; support = 6156
Value =  1 : precision =  0.60 ; recall =  0.03 ; f1-s = 0.06 ; support = 351
----------------------------------------------

You can run the following cells if you want to save the previous report on a text file

In [ ]:
# You can change this accordingly to your wish
output_file_path = '../models/models_files/'
# You can pick the format for the score file name
output_file_name = 'results_pipeline.txt'
output_file_full_name = output_file_path + output_file_name

In [ ]:
# Iterate and print on file
with open(output_file_full_name, "w") as text_file:
    print('-----------------------------------------------------------------------------------------', file=text_file)
    print("Date and time when this report was generated: ", datetime.datetime.now(), file=text_file)
    for ind_1 in range(y_pred.shape[1]):
        print('-----------------------------------------------------------------------------------------', file=text_file)
        print('Label = ', category_names[ind_1], file=text_file)
        c_rep = classification_report(y_test[:,ind_1], y_pred[:,ind_1], output_dict=True, zero_division=0)
        kk = list(c_rep.keys())
        for ind_2 in range(len(c_rep) - 3):
            print('Value = ', kk[ind_2], ': precision = ', "{:.2f}".format(c_rep[kk[ind_2]]['precision']),
                '; recall = ', "{:.2f}".format(c_rep[kk[ind_2]]['recall']),
                '; f1-s =', "{:.2f}".format(c_rep[kk[ind_2]]['f1-score']),
                '; support =', c_rep[kk[ind_2]]['support'], file=text_file)

### 7. Export your model as a pickle file

If you have just now created the pipeline, save it and the data in a pickle file

In [24]:
# Create dictionary for pickle file
model_dict_2 = {'X_train':X_train,
                    'y_train':y_train,
                    'X_test':X_test,
                    'y_test':y_test,
                    'model':model}

In [25]:
# Save dictionary
filename = '../models/models_files/pipeline_trained_model.pkl'
pickle.dump(model_dict_2, open(filename, 'wb'))

### 8. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Find what are the parameters of the pipeline
model.get_params()

In [ ]:
# Define grid search parameters
parameters = {
#         'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
#         'vect__max_features': (None, 5000, 10000),
#         'tfidf__use_idf': (True, False),
#         'clf__estimator__n_estimators': [50, 100, 200],
#         'clf__estimator__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(model, param_grid=parameters)

**NOTE:** Running the grid search on the full parameter space can take a **_lot_** of time

In [ ]:
# train classifier, keeping track of time
t0= time.clock()

cv.fit(X_train, y_train);

t1 = time.clock() - t0

###  9. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.